# Example AML Deployment Notebook

The following notebook provides an example for deploying an FFModel solution to an Azure Machine Learning (AML) Managed Endpoint. Executing this notebook will prepare an AML Managed Endpoint with an example FFModel Solution.

In the cell below, we have an `solution_config_path` which is the path to an solution configuration yaml describing a single FFModel LLM-based solution that we'd like to deploy for inference. This solution will be run in the context of the FFModel environment configuration described in the file located at `enviornment_config_path`. We will also name the deployment `nl2python` as it is an inferencing solution for converting natural language prompts to python code.  

In [ ]:
solution_config_path = "./nl2python_solution.yaml"
environment_config_path = "~/.ffmodel"
deployment_name = "nl2python"

The cell below triggers the actual deployment of the FFModel solution to the FFModel AML intance. We will create an AML Managed Endpoint called `FFModelSolutionEndpoint` and on it there will be a deployment named `nl2python` set to 100% of the traffic. The solution on this deployment will correspond to the solution config we provided for deployment. The deployment will be backed by a virtual machine with the SKU type `sku_type` (You can find a list of [available SKU types here](https://learn.microsoft.com/en-us/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list). To see the SKU quota available on your FFModel AML instance, follow the [guidance here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-manage-quotas#view-your-usage-and-quotas-in-the-azure-portal).).

Note that this step will take a few minutes to execute and progress can be monitored here or in the AML Studio.

In [ ]:
from ffmodel.core.aml import deployment_orchestrator

deployment_orchestrator.deploy_solution(
    solution_config_path=solution_config_path,
    environment_config_path=environment_config_path,
    endpoint_name="FFModelSolutionEndpoint",
    deployment_name=deployment_name,
    deployment_description=f"Sample '{deployment_name}' deployment to FFModel Solution Endpoint",
    sku_type="Standard_DS1_v2", # default: Standard_DS2_v2
    traffic_volume=100, # First deployment must have 100% of traffic; future deployments can have traffic <= 100%
)

## Testing Your Deployed FFModel Solution

To test your deployed solution, you can reference the Endpoint's "Test" tab in the AML Studio (more information [here](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-managed-online-endpoint-studio#test)). Alternatively, you can follow the "Consume" tab in the AML Studio and use the sample code there to test your deployed solution. The code in the next few cells is taken from the AML "Consume" tab and repurposed to generate a response to a simple "hello world" example request.

In [ ]:
# Endpoint Url
url = 'https://ffmodelsolutionendpoint.southcentralus.inference.ml.azure.com/score'

In [ ]:
# Replace this with the primary/secondary key or AMLToken for the endpoint
api_key = '<API-KEY>'

In [ ]:
# Sample "Hello World" Request Data
request_data = {
    "user_nl": "print 'hello world' to the console"
}

In [ ]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

body = str.encode(json.dumps(request_data))

if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")

# The azureml-model-deployment header will force the request to go to a specific deployment.
# Remove this header to have the request observe the endpoint traffic rules
headers = {
    'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key),
    'azureml-model-deployment': deployment_name
}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read().decode("utf8", "ignore")
    print(result)
except urllib.error.HTTPError as error:
    # Print the headers - they include the request ID and the timestamp, which are useful for debugging the failure
    print("The request failed with status code: " + str(error.code))
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))